In [1]:
import numpy as np


def compute_pagerank(G, alpha=0.85, q=None, max_iter=100, tol=1.0e-6):
    n = G.shape[0]
    if q is None:
        q = np.ones(n) / n
    p = np.ones(n) / n
    row_sums = G.sum(axis=1)
    G_normalized = G / row_sums[:, np.newaxis]

    for _ in range(max_iter):
        p_new = alpha * G_normalized.T.dot(p) + (1 - alpha) * q
        if np.linalg.norm(p_new - p, ord=1) <= tol:
            break

        p = p_new

    return p

def compute_pagerank_matrix(adjacency_matrix, alpha=0.85, strategy_x=None):
    """
    Computes the PageRank vector for an entire graph represented by an adjacency matrix.

    :param adjacency_matrix: Adjacency matrix representing the graph.
    :param alpha: Damping factor used in PageRank.
    :param strategy_x: Strategy matrix indicating which edges are retained (1) or not (0).
    :return: The PageRank vector for the entire graph.
    """
    if strategy_x is None:
        # If no strategy is provided, use the adjacency matrix itself
        strategy_x = adjacency_matrix

    n = adjacency_matrix.shape[0]  # Number of vertices

    # Normalize the adjacency matrix to get the transition probability matrix
    M = adjacency_matrix / np.sum(adjacency_matrix, axis=1, keepdims=True)
    print(M)

    # Apply strategy to the transition probability matrix
    M = M * strategy_x

    print(M)

    # Initialize the PageRank vector with a uniform distribution
    pi = np.ones(n) / n

    # Compute the PageRank vector iteratively
    for _ in range(1000):  # Hardcoded max_iter
        new_pi = alpha * np.matmul(M, pi) + (1 - alpha) / n
        if np.linalg.norm(new_pi - pi) < 1e-6:  # Hardcoded tolerance
            break
        pi = new_pi

    return pi

In [2]:
# Adjacency matrix
adjacency_matrix = np.array([
    [0, 1, 1, 1, 1],
    [1, 0, 1, 1, 0],
    [1, 0, 0, 1, 0],
    [1, 0, 1, 0, 0],
    [1, 0, 0, 0, 0]
])

n = adjacency_matrix.shape[0]
pi_T = compute_pagerank(adjacency_matrix)
print("PageRank:", pi_T)


# L_w^{in}
incoming_links = np.sum(adjacency_matrix, axis=0)
outgoing_links = np.sum(adjacency_matrix, axis=1)

print(incoming_links)
print(outgoing_links)

L_w_in = np.zeros(n)
for i in range(n):
    for j in range(n):
        if adjacency_matrix[j][i] == 1:  # 指向i的节点
            L_w_in[i] += outgoing_links[j]
    # print(incoming_links[i], L_w_in[i])
    L_w_in[i] = incoming_links[i] / L_w_in[i]

# print(L_w_in)


# N_k/N_T
keyword_frequency = np.array([1.042, 1.032, 1.030, 1.023, 1.023])

# T
traffic = np.array([7400, 2500, 2400, 1600, 1100])
T_min = np.min(traffic)
T_max = np.max(traffic)
T_hat = 0.1 + (traffic - T_min) / (T_max - T_min)

pi_new = pi_T * L_w_in * (1 + keyword_frequency) + T_hat

print(pi_T)
print(pi_new)

PageRank: [0.33961209 0.10216768 0.22802627 0.22802627 0.10216768]
[4 1 3 3 1]
[4 3 2 2 1]
[0.33961209 0.10216768 0.22802627 0.22802627 0.10216768]
[1.44674394 0.3741234  0.46064699 0.3331308  0.1516713 ]
